# __Enriquecimiento de la data__

## Librerias

In [1]:
%matplotlib inline
import pandas as pd

## Cargar datos

In [2]:
compras_df = pd.read_csv('../data_out/clean_compras.csv')
compras_df.dtypes

NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

## Expansion de los datos que tenemos

Convertimos la variable FECHA a tipo tiempo

In [5]:
compras_df['FECHA'] = pd.to_datetime(compras_df['FECHA'])
compras_df.dtypes

NUM BEN                  int64
BENEFICIARIO            object
RFC                     object
FACTURA                 object
CONTRATO                object
CONCEPTO                object
TIPO                    object
# OP                   float64
FECHA           datetime64[ns]
IMPORTE                float64
SHEET                   object
dtype: object

Obtener fecha en formato dia, mes, ano. Esto nos puede ayudar a modelar una clasificacion del comportamiento de otras variables en el tiempo.

In [6]:
compras_df['DIA'] = compras_df['FECHA'].dt.day
compras_df['MES'] = compras_df['FECHA'].dt.month
compras_df['YEAR'] = compras_df['FECHA'].dt.year

In [7]:
compras_df.head()

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,YEAR
0,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS AF8458-158008-3972,OT. 20204373,REEMPLAZO E INSTALACION DE AMORTIGUADORES (DEL...,TR,1019110.0,2020-12-11,14911.80,GASTOS VARIOS,11,12,2020
1,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6916-158008-4041,OT. 20204425,REEMPLAZO E INSTALACION DE OCHO BIRLOS Y NUEVE...,TR,1019110.0,2020-12-11,2528.80,GASTOS VARIOS,11,12,2020
2,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6917-158008-3975,OT. 20204376,REEMPLAZO DE DOS RINES (17). PATRIMONIO 80383...,TR,1019110.0,2020-12-11,8500.02,GASTOS VARIOS,11,12,2020
3,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7004-158008-3971,OT. 20204374,REEMPLAZO DE DOS RINES (14). PATRIMONIO 8007...,TR,1019110.0,2020-12-11,2400.01,GASTOS VARIOS,11,12,2020
4,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7137-158008-4554,OT. 20205130,REEMPLAZO E INSTALACION DE RIN 14. PATRIMONIO ...,TR,299001496.0,2020-12-23,1537.00,GASTOS VARIOS,23,12,2020


## Proyeccion entre variables 

In [8]:
beneficiario_sales = compras_df['BENEFICIARIO'].value_counts().to_dict()

In [9]:
# mostrar los primeros 5 valores de beneficiario_sales
list(beneficiario_sales.keys())[0:5]

['CAMARA MEXICANA DE LA INDUSTRIA DE LA CONSTRUCCION',
 'AUTOKAM REGIOMONTANA,S.A.DE C.V.',
 'CAR ONE MONTERREY, S.A. DE C.V.',
 'PEREZ RIOS RAMIRO',
 'S.I.M.E.P.R.O.D.E.']

In [11]:
# proyeccion de beneficiarios por ventas
compras_df['BENEFICIARIO_SALES'] = compras_df['BENEFICIARIO'].apply(lambda x: beneficiario_sales[x])
compras_df.BENEFICIARIO_SALES.head()

0    13
1    13
2    13
3    13
4    13
Name: BENEFICIARIO_SALES, dtype: int64

In [12]:
# proyeccion de ventas por tipo de hoja
sheet_mean_compras = compras_df.groupby('SHEET')['IMPORTE'].mean()
sheet_mean_compras

SHEET
COMUNICACION              109705.343662
CONTRATISTAS Y FDO FED    474566.765787
GASTOS VARIOS             881659.731797
SERV PPROF                229592.166667
SERV. PERS.                59623.405217
Name: IMPORTE, dtype: float64

In [13]:
compras_df['IMPORTE_GASTO_PROMEDIO'] = compras_df['SHEET'].apply(lambda x: sheet_mean_compras[x])
compras_df.IMPORTE_GASTO_PROMEDIO.head()

0    881659.731797
1    881659.731797
2    881659.731797
3    881659.731797
4    881659.731797
Name: IMPORTE_GASTO_PROMEDIO, dtype: float64

## Traer informacion externa

Vamos a utilizar una herramienta que nos permite a traves del nombre del beneficiario, verificar si tiene pagina en Linkedin.

La herramienta se llama _PhantomBuster_. Una vez pasado los beneficiarios, obtenemos un archivo csv con una columna indicativa sobre el resultado encontrado. En algunos casos, no encuentra el resultado, lo que indica que tal beneficiario no tiene pagina en Linkedin. Con esta informacion podemos crear una nueva variable en el df

In [14]:
# archivo con los nombres de los beneiciarios
pd.DataFrame(compras_df['BENEFICIARIO'].unique()).to_csv('../data_out/unique_beneficiarios.csv', index=None)

Este archivo, se le pasa a la herramienta y esta nos devuelve un csv con la informacion requerida. Vamos a cargar el archivo llamado _linkedin_mty.csv_

In [15]:
linkedin_mty = pd.read_csv('../data_in/linkedin_mty.csv')
linkedin_mty

,url,description,title,query,timestamp,error
0,https://www.linkedin.com/in/rosa-lorena-valdez...,Ve el perfil de Rosa Lorena Valdez Miranda en ...,Rosa Lorena Valdez Miranda - Analista de Deuda...,Rosa Lorena Valdez Miranda,2021-03-08T13:06:25.587Z,NaN
1,https://www.linkedin.com/in/roque-ya%C3%B1ez-r...,"Ve el perfil de Roque Yañez Ramos en LinkedIn,...",Roque Yañez Ramos - Síndico Primero - Gobierno...,Roque Yañez Ramos,2021-03-08T13:06:52.599Z,NaN
2,https://www.linkedin.com/in/perla-cordova-vill...,Ve el perfil de Perla Cordova Villegas en Link...,Perla Cordova Villegas - Profesor de Tiempo Co...,Laura Perla Cordova Rodriguez,2021-03-08T13:07:18.940Z,NaN
3,https://www.linkedin.com/in/lauren-coronado-b3...,"View Lauren Coronado’s profile on LinkedIn, th...",Lauren Coronado - Substitute Teacher - IRVING ...,Rosa Ofelia Coronado Flores,2021-03-08T13:07:45.390Z,NaN
4,https://www.linkedin.com/in/alvaro-palomo,Ve el perfil de Alvaro Palomo Osuna en LinkedI...,Alvaro Palomo Osuna - Jefe de proyecto .NET - ...,Alvaro Flores Palomo,2021-03-08T13:08:11.744Z,NaN
...,...,...,...,...,...,...
315,https://www.linkedin.com/in/fernando-mijangos-...,"Fernando Mijangos | Mérida, Yucatán, México | ...",Fernando Mijangos - Gerente Administrativo - D...,MIJANGOS DE JESUS FERNANDO,2021-07-23T19:37:32.917Z,NaN
316,https://www.linkedin.com/in/jos%C3%A9-arturo-m...,"José Arturo Moreno | México, México | 500+ con...","José Arturo Moreno - México, México | Perfil p...",MORENO GARCIA JOSE ARTURO,2021-07-23T19:37:57.780Z,NaN
317,https://www.linkedin.com/in/sebastian-hernande...,"Sebastian Hernandez Sanchez | Bluffton, South ...",Sebastian Hernandez Sanchez - Law Clerk - Devi...,SANCHEZ HERNANDEZ SEBASTIAN,2021-07-23T19:38:23.951Z,NaN
318,https://www.linkedin.com/in/fredy-marcelo-diaz...,Ve el perfil de Fredy Marcelo Diaz Segovia en ...,Fredy Marcelo Diaz Segovia - Universidad Santo...,SEGOVIA PAEZ MARCELO,2021-07-23T19:38:49.698Z,NaN
